# Using NBuilder to Speed Up Test Object Creation

In [1]:
#r "nuget:NBuilder,6.1.0"

In [1]:
enum CurveType { Forwards, Settle, Stats }
enum CompanyType { Internal, External, Exchange }
class Company {
    public int Id { get; set; }
    public string Name { get; set; }
    public CompanyType Type {get; set; }
    public override string ToString() => $"{Name} - {Type}";
}
class TimeSeriesInstance {
    public DateTime ObservationDate { get; set; }
    public decimal Value { get; set; }
    public override string ToString() => $"{ObservationDate:d} - {Value:n2}";
}
class Curve {
    public int ID { get; set; }
    public CurveType Type { get; set; }
    public int ProductId { get; set; }
    public Company[] Publishers { get; set; }  = Array.Empty<Company>();
    public IList<TimeSeriesInstance> Instances { get; set; } = new List<TimeSeriesInstance>();
    public IDictionary<string, string> Properties { get; set; } = new Dictionary<string, string> { 
        ["Frequency"] = "Daily"
    };

    public string GetKey() => $"{ProductId}:{Publishers.FirstOrDefault()?.Name}:{Type.ToString()}";

    /*
    public string Name {
        get {
            var frequency = Properties["Frequency"];
            var mainPublisher = Publishers[0].Id;
            var type = Type switch {
                CurveType.Forwards => "FWD",
                _ => "OTHER"
            };

            return $"{frequency} {type} by company {mainPublisher}";
        }
    }
    */
}

## Create a List

In [1]:
using FizzWare.NBuilder;

var curves = Builder<Curve>.CreateListOfSize(5).Build();
curves

index,ID,Type,ProductId,Publishers,Instances,Properties
0,1,Forwards,1,[ ],[ ],"[ [Frequency, Daily] ]"
1,2,Settle,2,[ ],[ ],"[ [Frequency, Daily] ]"
2,3,Stats,3,[ ],[ ],"[ [Frequency, Daily] ]"
3,4,Forwards,4,[ ],[ ],"[ [Frequency, Daily] ]"
4,5,Settle,5,[ ],[ ],"[ [Frequency, Daily] ]"


## Create a List - Control Values

In [1]:
using FizzWare.NBuilder;

var curves = Builder<Curve>.CreateListOfSize(5)
    .All()
        .With(c => c.ProductId = 20)
    .TheFirst(2)
        .With(c => c.Type = CurveType.Forwards)
    .TheNext(3)
        .With(c => c.Type = CurveType.Stats)
    .TheLast(3)
        .With(c => c.Properties["Frequency"] = "Weekly")
    .Build();
curves

index,ID,Type,ProductId,Publishers,Instances,Properties
0,1,Forwards,20,[ ],[ ],"[ [Frequency, Daily] ]"
1,2,Forwards,20,[ ],[ ],"[ [Frequency, Daily] ]"
2,3,Stats,20,[ ],[ ],"[ [Frequency, Weekly] ]"
3,4,Stats,20,[ ],[ ],"[ [Frequency, Weekly] ]"
4,5,Stats,20,[ ],[ ],"[ [Frequency, Weekly] ]"


## Populate Some List Properties from Other Generated Lists

In [1]:
using FizzWare.NBuilder;

var companies = Builder<Company>.CreateListOfSize(10).Build();
var timeSeries = Builder<TimeSeriesInstance>.CreateListOfSize(100).Build();

var curves = Builder<Curve>.CreateListOfSize(3)
    .All()
        .With(c => c.Publishers = new[] { Pick<Company>.RandomItemFrom(companies) })
        .With(c => c.Instances = Pick<TimeSeriesInstance>.UniqueRandomList(10).From(timeSeries).ToList())
    .Build();

curves

## Control Incremental Value Generation

In [1]:
using FizzWare.NBuilder;

var dateSeqGen = new SequentialGenerator<DateTime> {
    Direction = GeneratorDirection.Ascending,
    IncrementDateBy = IncrementDate.Day,
};
dateSeqGen.StartingWith(new DateTime(2010, 3, 1));

var timeSeries = Builder<TimeSeriesInstance>.CreateListOfSize(10)
    .TheFirst(5)
    .With(x => x.ObservationDate = dateSeqGen.Generate())
    .Build();

timeSeries

## Create Hierarchies